In [1]:
import pandas as pd

# Create an input dataset containing only the transcript column and stripping the PGN header

def transform_text(text):
    new_text = text.split('\n\n')[1].strip()
    return new_text[:1023]

input_file = 'full_dataset.csv'

df = pd.read_csv(input_file, usecols=['transcript'])

df['transcript'] = df['transcript'].apply(transform_text)

df.to_csv('input_dataset.csv', index=False)



In [2]:
import pandas as pd

df = pd.read_csv('input_dataset.csv', usecols=['transcript'])
df['length'] = df['transcript'].apply(len)

df.sort_values(by='length', inplace=True)
df.to_csv('input_dataset.csv', index=False)

In [3]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('input_dataset.csv')

# Randomly sample 10000 rows
sampled_df = df.sample(n=10000)

# Sort the sampled DataFrame by the length of the 'transcript' column
sampled_df.sort_values(by='transcript', key=lambda x: x.str.len(), ascending=True, inplace=True)

# Save the sampled and sorted DataFrame to a new CSV file
sampled_df.to_csv('test_input_dataset.csv', index=False)


In [4]:
import pandas as pd
import random

# Read the CSV file
df = pd.read_csv('test_input_dataset.csv')

# Prepare the new dataset for blocks
blocks = []
remaining_games = df['transcript'].tolist()

# Block size limit
block_size = 1024

while remaining_games:
    block = ';'
    # Select the next game
    next_game = remaining_games.pop()
    block += next_game
    while len(block) < block_size and remaining_games:
        next_game = remaining_games.pop(0)
        block += ';' + next_game
        if len(block) > block_size:
            # If the game makes the block too long, re-add it to the dataset
            if len(df) > 100:
                remaining_games.insert(99, next_game)
            else:
                remaining_games.insert(0, next_game)
            break

    if len(block) >= block_size:
        # Add the block to the blocks list
        blocks.append(block[:block_size])

# Create a new DataFrame for the blocks
blocks_df = pd.DataFrame(blocks, columns=['transcript'])

# Save the blocks to a new CSV file
blocks_df.to_csv('chess_blocks.csv', index=False)

# Save the remaining games to the original file
# remaining_df = pd.DataFrame(remaining_games, columns=['transcript'])
# remaining_df.to_csv('input_dataset.csv', index=False)


In [5]:
import pandas as pd

df = pd.read_csv('chess_blocks.csv')
df['length'] = df['transcript'].apply(len)
print(df['length'].describe())

count    5869.000000
mean     1023.912592
std         6.696306
min       511.000000
25%      1024.000000
50%      1024.000000
75%      1024.000000
max      1024.000000
Name: length, dtype: float64
